# Visualization in python
There are several [visualization](https://en.wikipedia.org/wiki/Edge_detection) tools for 3D imagery have been developed with/for Python, for example
Matplotlib (Hunter, 2007), Mayavi (Ramachandran & Varoquaux, 2011), the [ipyvolume](https://github.com/maartenbreddels/ipyvolume/), the yt Project
(Turk et al., 2010), and [ITK](https://itk.org/) (Johnson, McCormick, Ibanez 2015). 

The 3 main challenges of available tools are: 
- working with large volumes: image sizes
- complexity of APIs: how many lines for simple rendering
- compatibility to ipynb as much of the prototyping is within Jupyter notebooks

This lesson will illustrate tools that work with large datasets by exploring 3D projections as well as data reduction techniques enabled through `scikit-image` functions. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#from ipywidgets import interact

#from scipy import ndimage as ndi

from skimage import exposure, io, measure
                   # morphology, restoration, segmentation, transform,
                    #util)

%matplotlib inline

Let's read the original image and the labeled result from previous lesson:

In [ ]:
input_filename="data/cells.tif"
orig = io.imread(input_filename)
equalized = exposure.equalize_hist(orig)

In [ ]:
from skimage.external import tifffile
inputfile = 'data/cells_interior_labels.tiff'
relabeled = tifffile.imread(inputfile)

Let's create a simple visualizer

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

def slicer(z):
    plt.imshow(relabeled[z,:,:], cmap='gray')

interact(slicer, z=widgets.IntSlider(min=0,max=60,step=1,value=5));

Large datasets are a gift and a curse: while high-resolution imaging brings details about microstructures, the file sizes can quickly became an obstacle to analysis, particularly at your laptop. Here are some tips on how to proceed with your first explorations of your imagesets

In [ ]:
# MaxProjection:
        
IM_MAX = relabeled[0]
for n, image in enumerate(relabeled): #ndown here for low variation on Z
    IM_MAX = np.maximum(IM_MAX,image)
plt.imshow(IM_MAX)

## Surface of 3D objects
`skimage.measure.regionprops` **==>** `skimage.measure.marching_cubes` **==>** `skimage.measure.mesh_surface_area`

Perimeter measurements are not computed for 3D objects. The 3D extension of perimeter is surface area. We can measure the surface of an object by generating a surface mesh with 
- `skimage.measure.marching_cubes` returns 2 arrays: spatial coordinates for V unique mesh vertices and F faces that define triangular faces via referencing vertex indices from V. This algorithm specifically outputs triangles, so each face has exactly three indices.
- Marching cubes: extract a polygonal mesh of an isosurface from a three-dimensional discrete scalar field (voxels); basic steps are: divide the input volume into discrete set of cubes; each cube contains a piece of a given isosurface; a triangular mesh approximates the behavior of the trilinear interpolant in the interior cube.

In [ ]:
selected_cell = 3

regionprops = measure.regionprops(relabeled.astype('int'))
# skimage.measure.marching_cubes expects ordering (row, col, pln)
volume = (relabeled == regionprops[selected_cell].label).transpose(1, 2, 0)

verts_px, faces_px, _, _ = measure.marching_cubes_lewiner(volume, level=0, spacing=(1.0, 1.0, 1.0))

Now we will compute the surface area of the mesh with `skimage.measure.mesh_surface_area`.

In [ ]:
surface_area_pixels = measure.mesh_surface_area(verts_px, faces_px)
print("Surface area (total pixels): {:0.2f}".format(surface_area_pixels))

Remember that the cell data comes from an anisotropic acquisition, then we need to customize spacing:

In [ ]:
# The microscope reports the following spacing:
original_spacing = np.array([0.2900000, 0.0650000, 0.0650000])
print(f'* Microscope original spacing: {original_spacing}')

# We downsampled each slice 4x to make the data smaller
rescaled_spacing = original_spacing * [1, 4, 4]
print(f'* Microscope after rescaling images: {rescaled_spacing}')

# Normalize the spacing so that pixels are a distance of 1 apart
spacing = rescaled_spacing / rescaled_spacing[2]
print(f'* Microscope normalized spacing: {spacing}')

In [ ]:
verts, faces, _, _ = measure.marching_cubes_lewiner(volume, level=0, spacing=tuple(spacing))
surface_area_actual = measure.mesh_surface_area(verts, faces)

print("Surface area (total pixels): {:0.2f}".format(surface_area_pixels))
print("Surface area (actual): {:0.2f}".format(surface_area_actual))

In [ ]:
plt.plot(verts[:,0],verts[:,1],'b.')

# MPL_toolkit
It ships with Matplotlib and thus may be a lighter weight solution for some use cases. Provide basic 3D plotting of: 
- scatter, 
- surf, 
- line, 
- mesh

Not the fastest or most feature complete 3D library out there. Check out the mplot3d [tutorial](https://matplotlib.org/tutorials/toolkits/mplot3d.html#sphx-glr-tutorials-toolkits-mplot3d-py) for more information. mplot3d supplies an axes object that can create a 2D projection of a 3D scene.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
#method for creating a 3D axes object
ax = fig.gca(projection='3d')

ax.scatter(verts[:,0],verts[:,1],verts[:,2])

plt.show()


## Exercise 1:
a) Select a different cell, e.g. selected_cell = 1, and calculate the actual surface area

b) explore parameters to improve the scatter plot, e.g., label and colors

c) find the largest connected component

In [ ]:
#%load solution/03_solution01_a.py

In [ ]:
#%load solution/03_solution01_b.py

In [ ]:
# %load solution/03_solution01_c.py

Is it possible to create a better projection that that? Yes, the volume (polyhedron) can be visualized using the mesh, a set of polygons created by `Poly3DCollection` determined by the vertexes and faces.

In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

selected_cell = 3
fig = plt.figure(figsize=(10, 10))

#method for creating a 3D axes object
ax = fig.add_subplot(111, projection='3d')

mesh = Poly3DCollection(verts_px[faces_px])
#makes edges visible
mesh.set_edgecolor('black') 
ax.add_collection3d(mesh)

#retrieve spatial coord of selected cell
min_pln, min_row, min_col, max_pln, max_row, max_col = regionprops[selected_cell].bbox

#requires setting the region of interest
ax.set_xlim(min_row, max_row)
ax.set_ylim(min_col, max_col)
ax.set_zlim(min_pln, max_pln)
plt.show()

# ITK and ITKwidgets in itkwidgets
- VTK is a powerful scientific visualization library with Python bindings, and speedy C++ backend. The library itkwidgets provides interactive ipywidgets to visualize images in 2D and 3D; 
- ITKwidgets are Jupyter widgets to visualize images, point sets, and meshes;
- More at: [[repo]](https://github.com/InsightSoftwareConsortium/itkwidgets) [[examples]](https://itk.org/ITKExamples/src/index.html) [[docs]](https://itkpythonpackage.readthedocs.io/en/latest/Quick_start_guide.html)

In [ ]:
import itk
from itkwidgets import view

In [ ]:
#Reformat the numpy array into an itk matrix
image = itk.GetImageFromArray(relabeled.astype(np.int16))
view(image, slicing_planes=True)

In [ ]:
#Reading with itk
input_filename="data/bead_pack_artifacts_small.tif"
image = itk.imread(input_filename)
view(image, cmap='Cold and Hot', slicing_planes=True,gradient_opacity=0.4)

In [ ]:
#convert to nnumpy array
image_np = itk.GetArrayFromImage(image)
print(f"* Data shape: {image_np.shape}")
print(f"* Data type:  {image_np.dtype}")
print(f"* Data range: ({image_np.min()}, {image_np.max()})")

## Data reduction using slicing
- The syntax ::2 easily extract the elements of a list that have even indexes; in other words, you will retrieve every other slice

In [ ]:
from skimage.filters import threshold_isodata
image_np=image_np[::2,::2,::2]
image_np = image_np>threshold_isodata(image_np)
image_itk = itk.GetImageFromArray(image_np.astype(np.int16))
view(image_itk, cmap='Cold and Hot',rotate=True, gradient_opacity=0.4)

## Exercise 2. 
- Explore the parameters in `view` and select roi
- Visualize another dataset, e.g. data/mriBrain.tif

In [ ]:
# This image has anisotropic spacing

input_filename="data/mriBrain.tif"
image = itk.imread(input_filename)
view(image, cmap='Cold and Hot', slicing_planes=True,gradient_opacity=0.4)

In [ ]:
print(image.GetSpacing())
image.SetSpacing([1,1,5])
print(image.GetSpacing())
view(image, cmap='Cold and Hot', slicing_planes=True,gradient_opacity=0.4)

In [ ]:
#view(image, cmap='Cold and Hot', slicing_planes=True,gradient_opacity=0.4)

## Going beyond

[1] Segmentation of 3-D tomography images with Python: http://emmanuelle.github.io/segmentation-of-3-d-tomography-images-with-python-and-scikit-image.html

[2] ITKwidgets examples: https://github.com/InsightSoftwareConsortium/itkwidgets/tree/master/examples

[3] Image processing with Dask arrays: https://dask-image.readthedocs.io/en/latest/